In [25]:
from math import *
from typing import List, Callable

import numpy as np

def machine_epsilon(func=float):
    '''
    Вспомогательная функция для вычисления машинного нуля
    '''
    machine_epsilon = func(1)
    while func(1)+func(machine_epsilon) != func(1):
        machine_epsilon_last = machine_epsilon
        machine_epsilon = func(machine_epsilon) / func(2)
    return machine_epsilon_last

ME = machine_epsilon()*10

def eval_func(t):
    '''
    Интегрируемая функция
    '''
    def inner_func_LR(theta, phi):
        return (2*cos(theta)) / (1-(sin(theta)**2)*(cos(phi)**2))
    def inner_func_main(t, LR_func, theta, phi):
        return (1-exp((-t)*LR_func(theta, phi))) * cos(theta)*sin(theta)
    return lambda theta, phi: inner_func_main(t, inner_func_LR, theta, phi)

def eval_Lm(x, n):
    '''
    Вычисление полинома Лежандра n-ой степени
    '''
    res = None
    if n > 1:
        Lm0 = 1 #значение полинома 0-й степени
        Lm1 = x #значение полинома 1-й степени
        lim_n = 2
        while True:
            res = (
                ((2*lim_n-1)*x*Lm1 - (lim_n-1)*Lm0)
                /lim_n
            )
            Lm0 = Lm1
            Lm1 = res
            if lim_n == n:
                break
            lim_n += 1
        return res
    else:
        return 1 if pow == 0 else x


def eval_Lm_derivative(x, n):
    '''
    Вычисление производной полинома Лежандра
    '''
    L_first = eval_Lm(x, n-1)
    L_second = eval_Lm(x, n)
    return (n/(1-x**2))*(L_first - x*L_second)

def eval_Lm_roots(n):
    '''
    Вычисление заданного количества корней полинома Лежандра
    '''
    roots_res = []
    roots_init_approx = [
        cos(
            (pi*(4*i-1)) / (4*n+2)
        )
        for i in range(1, n+1)
    ]
    for x in roots_init_approx:
        Lm = 1
        while Lm > ME:
            Lm = eval_Lm(x, n)
            Lm_deriv = eval_Lm_derivative(x, n)
            x = x - Lm/Lm_deriv
        roots_res.append(x)
    return roots_res

'========================================================================='
'========================================================================='
'========================================================================='

def eval_At_sum(k):
    '''
    Вычисление суммы произведений Ai*ti
    '''
    return 2/(k+1) if k%2==0 else 0

def build_At_matrix_with_coef_sum(n, Lm_roots):
    '''
    Строим СЛАУ
    '''
    M = []
    for k in range(n):
        row = []
        for ti in Lm_roots:
            row.append(ti**k)
        row.append(eval_At_sum(k))
        M.append(row)
    return M

def eval_Ai_coefs(M):
    '''
    Решаем СЛАУ, находим коэффициенты Ai
    '''
    n = len(M)
    X = [0 for i in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            ratio = M[j][i]/M[i][i]
            for k in range(n+1):
                M[j][k] = M[j][k] - ratio*M[i][k]
    X[n-1] = M[n-1][n]/M[n-1][n-1]
    for i in range(n-2, -1, -1):
        X[i] = M[i][n]
        for j in range(i+1, n):
            X[i] = X[i] - M[i][j]*X[j]
        X[i] = X[i]/M[i][i]
    return X

def eval_Gauss_integration(func, a, b, N_nodes):
    '''
    Проводим численное интегрирование по формуле Гаусса
    '''
    T = eval_Lm_roots(N_nodes)
    A = eval_Ai_coefs(build_At_matrix_with_coef_sum(N_nodes, T))
    sum_Aiti = sum([Ai*func((a+b)/2+((b-a)/2)*ti) for (Ai, ti) in zip(A, T)])
    res = ((b-a)/2)*sum_Aiti
    return res

def simpson_integrate(function: Callable, a: float, b: float, n: int) -> float:
    """ интегрирование методом Симпсона """
    h, res = (b - a) / n, 0
    x = a
    while x < b:
        next_x = x + h
        res += function(x) + 4 * function(x + h * .5) + function(next_x)
        x = next_x
    return h / 6 * res

def simpson_integrate2(function: Callable, a: float, b: float, n: int) -> float:
    """ интегрирование методом Симпсона """
    h, res = (b - a) / n, 0
    x = a
    while x < b:
        next_x = x + h
        res += function(x) + 4 * function(x + h * .5) + function(next_x)
        x = next_x
    return h / 6 * res

def composite_integrate(f: Callable, a1: float, b1: float, a2: float, b2: float,
                        method_1: Callable, method_2: Callable, n1: int, n2: int) -> float:
    func = lambda y: method_1(lambda x: f(x, y), a1, b1, n1)
    return method_2(func, a2, b2, n2)


def function_integrator(f: Callable, a: float, b: float, c: float, d: float, n: int, m: int) -> float:
    return composite_integrate(f, a, b, c, d, eval_Gauss_integration, simpson_integrate, n, m)

def integrate(t: float, n: int, m: int) -> float:
    f = eval_func(t)
    return 4 / np.pi * function_integrator(f, 0, np.pi * .5, 0, np.pi * .5, n, m)


print(integrate(0.808, 4, 4))
print(integrate(1.5, 11, 10))
t = 0.05
N = 10
M = 10
for i in range(20):
    print("t=", t, integrate(t, N, M))
    t += 0.5
print("t=", 10, integrate(10, N, M))

0.7528023808753337
0.9047127793191192
t= 0.05 0.09386138995766914
t= 0.55 0.6281067347324192
t= 1.05 0.8271123131965122
t= 1.55 0.9103405117431628
t= 2.05 0.9484860032977233
t= 2.55 0.9675738475408582
t= 3.05 0.9779745877556576
t= 3.55 0.9841086970754156
t= 4.05 0.9879874586762487
t= 4.55 0.9905879567350884
t= 5.05 0.9924167439854428
t= 5.55 0.9937533287161731
t= 6.05 0.9947610587235556
t= 6.55 0.9955403976972679
t= 7.05 0.9961559273862742
t= 7.55 0.9966507596850668
t= 8.05 0.9970546114473827
t= 8.55 0.9973885333903781
t= 9.05 0.997667791089854
t= 9.55 0.9979036822655599
t= 10 0.9980859847782937
